In [127]:
from owlready2 import *
import pandas as pd
import xml.sax
import xml.etree.ElementTree as ET
from rapidfuzz import process
# from rapidfuzz import fuzz
import rapidfuzz
import pprint
import time
import csv

----
# Loading data...

- loads the BRENDA ontology OWL file
    - loads the properties, classes and synonyms from BRENDA
    - classes information split into names and labels
    - properties split into annotation/object and also names and labels
- loads the body_site_terms from a csv of manually collated terms

In [5]:
# load the BTO ontology
onto_path.append("/data/ontology")
onto = get_ontology("http://purl.obolibrary.org/obo/bto.owl").load()
# print(onto.base_iri)

In [6]:
# load class information and properties from the ontology
class_names = [c.name for c in onto.classes()]
# account for missing class labels in the ontology
class_labels = [c.label[0] if len(c.label) > 0 else '' for c in onto.classes()]
class_synonyms = {c.name: c.hasExactSynonym + c.hasRelatedSynonym for c in onto.classes()}

num_synonyms = 0

for class_label in onto.classes():
    num_synonyms += len(class_label.hasRelatedSynonym) + len(class_label.hasExactSynonym)

print("Num synonyms: ", num_synonyms)
# properties_names contains all properties, including object and annotation properties
# names list contains ontology
object_properties_names = [o.name for o in onto.object_properties()]
object_properties_labels = [o.label[0] if len(o.label) > 0 else o.name for o in onto.object_properties()]

annotation_properties_names = [a.name for a in onto.annotation_properties()]
annotation_properties_labels = [a.label[0] if len(a.label) > 0 else a.name for a in onto.annotation_properties()]

properties_names = [p.name for p in onto.properties()]
properties_labels = [p.label[0] if len(p.label) > 0 else p.name for p in onto.properties()]

print("Number of classes: ", len(class_names))
# print(class_synonyms)

Num synonyms:  6210
Number of classes:  6569


In [7]:
# load the list of annotated body_site terms
body_site_terms = pd.read_csv("data/BioAnnotate_rh-body_site.csv", header=None)
# body_site_terms = body_site_terms[0].tolist()
body_site_terms = set(body_site_terms[0])

In [8]:
# print(onto.search(label="*bone*marrow*"))

for label in onto.search(label="*ileum*"):
    print(label.name, label.label)

print(onto.search_one(label="*ileum*").label)



BTO_0000620 ['ileum']
BTO_0005574 ['ileum smooth muscle']
BTO_0005575 ['ileum smooth muscle cell']
['ileum']


---- 
# Parsing the Biosamples files
- BioSamplesDictHandler looks at all the attributes for each biosample
    - uses a fuzzy search to find the attribute that has the highest similarity to the attribute names in the body_site_terms
- BioSamplesPackageHandler looks for 'useful' packages and finds the appropriate information
- BioSamplesAttributeHandler looks through all attributes of all samples and checks if the attribute is within the possible_tissue set

In [38]:
# load a single biosamples XML file. File 699 used due to smaller size
# SAX parser used to avoid memory issues - CAN IN THEORY LOAD ALL BIOSAMPLES IN ONE FILE
biosamples_path = "data/biosamples/biosample_random_samples.xml"
# biosamples_path = "data/biosamples/biosample_small.xml"

class BioSamplesDictHandler(xml.sax.ContentHandler):
    def __init__(self):
        super().__init__()
        self.attribute_name = ""
        self.num_samples = 0
        self.attribute_dict = {}
        self.sample_dict = {}
        self.biosample_id = ""

    def startElement(self, name, attrs):
        if name == "BioSample":
            self.num_samples += 1
            self.biosample_id = attrs["id"]

        elif name == "Attribute":
            self.attribute_name = attrs["attribute_name"]  
            
    def characters(self, content):
        if self.attribute_name != "":
            self.attribute_dict[self.attribute_name] = content
            self.attribute_name = ""


    def endElement(self, name):
        if name == "Attributes":
            if self.attribute_dict != {}:
                for key in self.attribute_dict.keys():
                    result = process.extractOne(key, body_site_terms, scorer=rapidfuzz.fuzz.ratio)
                    if result[1] > 87:
                        self.sample_dict[self.biosample_id] = (key, self.attribute_dict[key])
                        break
                    else:
                        self.sample_dict[self.biosample_id] = None
            self.attribute_dict = {}

    def endDocument(self):
        print("Number of possible tissue locations found: ", len([x for x in self.sample_dict.values() if x is not None]))
        print("Number of samples with at least one attribute: ", len(self.sample_dict))      
        # print the key value pairs in the sample dict that don't have None as the value in a list
        pprint.pprint([x for x in self.sample_dict.items() if x[1] is not None])


In [42]:
t1 = time.time()
parser = xml.sax.make_parser()
handler = BioSamplesDictHandler()
parser.setContentHandler(handler)
parser.parse(biosamples_path)
t2 = time.time()
print("Time taken: ", t2  -t1)

# using a direct lookup from body_site_terms, 45m 22.9s
# num_samples: 32526809
# num_tissue_samples: 6492612

# 17718
# 40839

# token sort ratio
# 3121

Number of possible tissue locations found:  3618
Number of samples with at least one attribute:  9918
[('24429',
  ('histological type',
   'Biopsy obtained in grossly disease unaffected region')),
 ('24727', ('histological type', 'healthy')),
 ('47520', ('body site', 'Primary Tumor')),
 ('52257', ('histological type', 'Serous Cystadenocarcinoma')),
 ('53145', ('histological type', 'Serous Cystadenocarcinoma')),
 ('53717', ('histological type', 'Serous Cystadenocarcinoma')),
 ('54997', ('histological type', 'Untreated primary (De Nova) GBM')),
 ('56953', ('body site', 'Blood Derived Normal')),
 ('61491', ('body site', 'Primary Tumor')),
 ('92151', ('body site', 'Primary Tumor')),
 ('93344', ('body site', 'Primary Tumor')),
 ('116303', ('body site', 'Blood')),
 ('123336', ('histological type', 'Serous Cystadenocarcinoma')),
 ('154619', ('tissue-type', 'spleen')),
 ('158248', ('organ', 'testis')),
 ('158377', ('organ', 'breast')),
 ('161214', ('organ', 'marrow')),
 ('174739', ('tissue-ty

---

In [27]:
# Code to get the list of package names and their required attributes

templates_path = "data/biosamples/templates_xml/"
biosamples_packages_xml_path = "data/biosamples/biosample_packages.xml"
packages_list = []

class TemplateNamesHander(xml.sax.ContentHandler):
    def __init__(self, templates_list) -> None:
        super().__init__()
        # self.templates = []
        self.is_name = False

    def startElement(self, name, attrs):
        if name == "Name":
            self.is_name = True

    def characters(self, content):
        if self.is_name:
            packages_list.append(content)
            self.is_name = False    
    
parser = xml.sax.make_parser()
parser.setContentHandler(TemplateNamesHander(templates_list=packages_list))
parser.parse(biosamples_packages_xml_path)


'''
- Reads the xml file for each package and extracts the mandatory attributes
- Mandatory attributes for a single package will be stored in a set
    - attributes where at least one in a group is required will be stored in a list

- All data will be stored in a dictionary where the key is the package name and the value is a list of mandatory attributes
'''
class MandatoryAttributeHandler(xml.sax.ContentHandler):
    def __init__(self, attributes_list) -> None:
        super().__init__()
        self.attributes_list = attributes_list
        self.is_mandatory_attribute = False
        self.group_name = None
        self.is_harmonized_name = False
        self.mandatory_groups = {}

    def startElement(self, name, attrs):
        if name == "Attribute":
            use = attrs["use"]
            if use == "optional":
                pass
            elif use == "mandatory":
                self.is_mandatory_attribute = True
            elif use == "either_one_mandatory":
                self.group_name = attrs["group_name"]
                self.is_mandatory_attribute = True
        elif name == "HarmonizedName":
            self.is_harmonized_name = True

    def characters(self, content):
        if self.is_mandatory_attribute and self.is_harmonized_name:
            if self.group_name is None:
                self.attributes_list.append(content)
            else:
                # group name is not None therefore the attribute is part of a group 
                # attribute name added to dictionary with group name as key
                if self.group_name in self.mandatory_groups:
                    self.mandatory_groups[self.group_name].append(content)
                else:
                    self.mandatory_groups[self.group_name] = [content]
                self.group_name = None
            
            self.is_mandatory_attribute = False
            self.is_harmonized_name = False

    def endDocument(self):
        for group in self.mandatory_groups.items():
            self.attributes_list.append(group)
        attribute_list = self.attributes_list


attributes_set = {}
for package in packages_list:
    with open("data/biosamples/templates_xml/" + package + ".xml") as f:
        attribute_list = []
        parser = xml.sax.make_parser()
        parser.setContentHandler(MandatoryAttributeHandler(attribute_list))
        parser.parse(f)
        attributes_set[package] = attribute_list

In [28]:
biosamples_path = "data/biosamples/biosample_random_samples.xml"
possible_tissue = {"cell_line", "cell_subtype", "cell_type", "host_tissue_sampled", "sample_type", "tissue", "host_anatomical_part", "isolation_source", "source_name", "source_type", "strain", "subclone", "subgroup"}
# possible_tissue = {"tissue"}

class BioSamplesPackageHandler(xml.sax.ContentHandler):
    def __init__(self, useful_packages, attributes_dict):
        super().__init__()
        self.useful_packages = useful_packages
        self.attributes_dict = attributes_dict
        self.is_package = False
        self.package_name = ""
        self.target_attributes = set()
        self.num_samples = 0
        self.biosample_id = ""
        self.attribute_name = ""
        self.sample_dict = {}

    def startElement(self, name, attrs):
        if name == "BioSample":
            self.biosample_id = attrs["id"]
            self.num_samples += 1
        elif name == "Package":
            self.is_package = True
        elif name == "Attribute" and "harmonized_name" in attrs:
                self.attribute_name = attrs["harmonized_name"]

    def characters(self, content):
        if self.is_package:
            self.package_name = content
            if self.package_name in self.useful_packages:
                self.target_attributes = possible_tissue.intersection(set(a for a in self.attributes_dict[self.package_name] if type(a) == str))
                if len(self.target_attributes) > 1:
                    print("more than one matching package: ", self.biosample_id)
            self.is_package = False

        if self.attribute_name != "":
            if self.attribute_name in self.target_attributes:
                self.sample_dict[self.biosample_id] = (self.attribute_name, content)
            self.attribute_name = ""

    def endDocument(self):
        pprint.pprint(self.sample_dict)
        print("Number of possible tissue locations found: ", len(self.sample_dict))
        


In [43]:
useful_packages = set()
with open("data/biosamples/useful_packages.txt", "r") as f:
    for package in f.readlines():
        useful_packages.add(package.strip())


t1 = time.time()
parser = xml.sax.make_parser()
handler = BioSamplesPackageHandler(useful_packages, attributes_set)
parser.setContentHandler(handler)
parser.parse(biosamples_path)
t2 = time.time()
print("Time taken: ", t2 - t1)


more than one matching package:  30414169
{'10037706': ('tissue', 'Whole blood'),
 '10039586': ('isolation_source', 'missing'),
 '10058440': ('isolation_source', 'food'),
 '10070699': ('tissue', 'cormel'),
 '10072908': ('tissue', 'Proximal jejunum'),
 '10080731': ('isolation_source', 'Hospital patients'),
 '10088371': ('tissue', 'Sputum'),
 '10092280': ('isolation_source', 'missing'),
 '10097534': ('isolation_source', 'human'),
 '10104985': ('isolation_source', 'Water column'),
 '10119081': ('tissue', 'tumor'),
 '10128932': ('tissue', 'blood'),
 '10142456': ('tissue', 'lung cancer'),
 '10148089': ('tissue', 'SKIN FIBROBLASTS'),
 '10148996': ('tissue', 'leaf'),
 '10162994': ('tissue', 'cortex'),
 '10163223': ('isolation_source', 'Drainage from GT site'),
 '10178733': ('isolation_source', 'upper respiratory tract'),
 '10179895': ('isolation_source', 'upper respiratory tract'),
 '10182641': ('isolation_source', 'missing'),
 '10218328': ('isolation_source', 'not applicable'),
 '10221719': 

---

In [9]:
class BioSamplesAttributeHandler(xml.sax.ContentHandler):
    def __init__(self, attributes_set, sample_dict) -> None:
        super().__init__()
        sample_dict = sample_dict
        self.attributes_set = attributes_set
        self.biosample_id = ""
        
        self.attribute_name = ""

    def startElement(self, name, attrs):
        if name == "BioSample":
            self.biosample_id = attrs["id"]
        elif name == "Attribute" and "harmonized_name" in attrs:
            attribute_name = attrs["harmonized_name"]
            if attribute_name in self.attributes_set:
                self.attribute_name = attribute_name

    def characters(self, content):
        if self.attribute_name != "":
            sample_dict[self.biosample_id] = (self.attribute_name, content)
            self.attribute_name = ""

    def endDocument(self):
        print("Number of possible tissue locations found: ", len(sample_dict))


In [10]:
biosamples_path = "data/biosamples/biosample_random_samples.xml"
possible_tissue = {"cell_line", "cell_subtype", "cell_type", "host_tissue_sampled", "sample_type", "tissue", "host_anatomical_part", "isolation_source", "source_name", "source_type", "strain", "subclone", "subgroup"}
sample_dict = {}

t1 = time.time()
parser = xml.sax.make_parser()
handler = BioSamplesAttributeHandler(possible_tissue, sample_dict)
parser.setContentHandler(handler)
parser.parse(biosamples_path)
t2 = time.time()
print("Time taken: ", t2 - t1)

pprint.pprint(sample_dict)

Number of possible tissue locations found:  6607
Time taken:  0.8625826835632324
{'1001155': ('cell_line', 'mesoderm'),
 '10037706': ('tissue', 'Whole blood'),
 '10039586': ('isolation_source', 'missing'),
 '10046016': ('tissue', 'Blood'),
 '10046210': ('tissue', 'Blood'),
 '10051706': ('tissue', 'Blood'),
 '10052104': ('tissue', 'Blood'),
 '10055770': ('tissue', 'adjacent normal tissue'),
 '10057704': ('source_name', 'mouse embryos'),
 '10057981': ('source_name', 'mouse embryos'),
 '10058440': ('isolation_source', 'food'),
 '10067453': ('tissue', 'Ileum'),
 '10070699': ('tissue', 'cormel'),
 '10072908': ('tissue', 'Proximal jejunum'),
 '10080520': ('sample_type', 'rumen fluid'),
 '10080731': ('isolation_source', 'Hospital patients'),
 '10085133': ('tissue', 'Liver'),
 '10088371': ('sample_type', 'Tissue_sample'),
 '10092280': ('isolation_source', 'missing'),
 '10095257': ('cell_type', 'bone marrow cells'),
 '10097534': ('isolation_source', 'human'),
 '10103116': ('source_name', 'Burki

- From basic code, directly looking at the attributes of each biosamples gives the most coverage in terms of possible tissue terms
- As expected, checking the package of the biosample provides a faster method of extracting the possible tissue terms
    - issues arise from the Generic.1.0 package from biosamples that have not been updated
- DictHandler that uses a fuzzy search is more than 5 times slower than other methods and does not give better coverage with current settings

----
# Attribute handler for biosample_set.xml
- same code as above but run on the whole set
- code writes the found information to a csv file in order to not overload memory

In [129]:
class BioSamplesAttributeHandler(xml.sax.ContentHandler):
    def __init__(self, attributes_set) -> None:
        super().__init__()
        self.attributes_set = attributes_set
        self.biosample_id = ""
        self.attribute_name = ""
        self.biosample_count = 0
        
        with open("data/biosamples/results/biosamples_tissue.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerow(["biosample_id", "tissue"])

    def startElement(self, name, attrs):
        if name == "BioSample":
            self.biosample_id = attrs["id"]
            self.biosample_count += 1
            if self.biosample_count % 1000000 == 0:
                print("Number of biosamples processed: ", self.biosample_count)
                
        elif name == "Attribute" and "harmonized_name" in attrs:
            attribute_name = attrs["harmonized_name"]
            if attribute_name in self.attributes_set:
                self.attribute_name = attribute_name

    def characters(self, content):
        if self.attribute_name != "":
            with open("data/biosamples/results/biosamples_tissue.csv", "a") as f:
                writer = csv.writer(f)
                writer.writerow([self.biosample_id, content])
            self.attribute_name = ""

    def endDocument(self):
        print("Finished writing to file")

In [133]:
biosamples_path = "data/biosamples/biosample_set.xml"
possible_tissue = {"cell_line", "cell_subtype", "cell_type", "host_tissue_sampled", "sample_type", "tissue", "host_anatomical_part", "isolation_source", "source_name", "source_type", "strain", "subclone", "subgroup"}

parser = xml.sax.make_parser()
handler = BioSamplesAttributeHandler(possible_tissue)
parser.setContentHandler(handler)
parser.parse(biosamples_path)

Number of biosamples processed:  1000000
Number of biosamples processed:  2000000
Number of biosamples processed:  3000000
Number of biosamples processed:  4000000
Number of biosamples processed:  5000000
Number of biosamples processed:  6000000
Number of biosamples processed:  7000000
Number of biosamples processed:  8000000
Number of biosamples processed:  9000000
Number of biosamples processed:  10000000
Number of biosamples processed:  11000000
Number of biosamples processed:  12000000
Number of biosamples processed:  13000000
Number of biosamples processed:  14000000
Number of biosamples processed:  15000000
Number of biosamples processed:  16000000
Number of biosamples processed:  17000000
Number of biosamples processed:  18000000
Number of biosamples processed:  19000000
Number of biosamples processed:  20000000
Number of biosamples processed:  21000000
Number of biosamples processed:  22000000
Number of biosamples processed:  23000000
Number of biosamples processed:  24000000
N

----
# Matching results to the BTO
- using the dictionary of possible tissue terms, a search can be carried out on the BTO to find the matching term


In [11]:
result_dict = {}
result_front_wildcard = {}
result_back_wildcard = {}
result_both_wildcard = {}

for sample_id, attribute in sample_dict.items():
    result = onto.search_one(label = "{}".format(attribute[1]), _case_sensitive = False)
    result_front = onto.search_one(label = "{}*".format(attribute[1]), _case_sensitive = False)
    result_back = onto.search_one(label = "*{}".format(attribute[1]), _case_sensitive = False)
    result_both = onto.search_one(label = "*{}*".format(attribute[1]), _case_sensitive = False)
    if result is not None:
        result_dict[sample_id] = (result.name, result.label[0])
    else:
        result_dict[sample_id] = (None, None)

    if result_front is not None:
        result_front_wildcard[sample_id] = (result_front.name, result_front.label[0])
    else:
        result_front_wildcard[sample_id] = (None, None)
    
    if result_back is not None:
        result_back_wildcard[sample_id] = (result_back.name, result_back.label[0])
    else:
        result_back_wildcard[sample_id] = (None, None)

    if result_both is not None:
        result_both_wildcard[sample_id] = (result_both.name, result_both.label[0])
    else:
        result_both_wildcard[sample_id] = (None, None)

# pprint.pprint(result_dict)
    

In [12]:
# # print the number of sammples that have a match
# positive_results = [(sample_id, result[1]) for sample_id, result in result_dict.items() if result != (None, None)]
positive_results = {sample_id: result[1] for sample_id, result in result_dict.items() if result != (None, None)}
print("Number of exact matches in the BTO: ", len(positive_results))


filtered_results = dict(filter(lambda x: x[1] == "blood", positive_results.items()))
print("Number of matches with 'Blood' in the BTO: ", len(filtered_results))
print(filtered_results)
# print({a for a in result_dict.values() if a is not None})

Number of exact matches in the BTO:  2133
Number of matches with 'Blood' in the BTO:  779
{'116303': 'blood', '182671': 'blood', '207804': 'blood', '209188': 'blood', '218984': 'blood', '259774': 'blood', '272252': 'blood', '282424': 'blood', '285590': 'blood', '299261': 'blood', '299409': 'blood', '300388': 'blood', '302005': 'blood', '302869': 'blood', '314075': 'blood', '320480': 'blood', '330926': 'blood', '358984': 'blood', '376017': 'blood', '378418': 'blood', '383282': 'blood', '398656': 'blood', '425799': 'blood', '426758': 'blood', '426891': 'blood', '428706': 'blood', '452333': 'blood', '461095': 'blood', '463605': 'blood', '479608': 'blood', '517051': 'blood', '518279': 'blood', '519059': 'blood', '519521': 'blood', '519689': 'blood', '520440': 'blood', '524175': 'blood', '528239': 'blood', '528273': 'blood', '529484': 'blood', '530324': 'blood', '531605': 'blood', '531794': 'blood', '533664': 'blood', '578669': 'blood', '604885': 'blood', '605938': 'blood', '608849': 'blood

----
# Gettting search results from NCBI
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=biosample&term=blood&retmax=100&usehistory=y
- Search results can be fetched from the NCBI website 
    - all biosample ids that match a given search term can be fetched
    - result returned as an XML file
- Comparison can be made results from the BioSamplesAttributeHandler and NCBI results to check for false positives

In [78]:
import requests
import keys
from time import sleep

In [110]:
base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
query = "esearch.fcgi?db=biosample&term=blood&retmax=100&usehistory=y"

url = base_url + query + "&api_key=" + keys.ncbi_key
response = requests.get(url)

assert response.status_code == 200, "Error: " + str(response.status_code)

# Gets the required values from the initial search response
root = ET.fromstring(response.content)
webenv = root.find("WebEnv").text
query_key = root.find("QueryKey").text
count = root.find("Count").text


ids = set()
for i in root.findall("IdList/Id"):
    ids.add(i.text)

retmax = 10000
for retstart in range(0, int(count), retmax):
    fetch_url = base_url + "efetch.fcgi?db=biosample&WebEnv=" + webenv 
    fetch_url += "&query_key=" + query_key + "&retstart=" + str(retstart) + "&retmax=" + str(retmax) 
    fetch_url += "&rettype=uilist&retmode=xml&api_key=" + keys.ncbi_key

    fetch_response = requests.get(fetch_url)
    assert fetch_response.status_code == 200, "Error: " + str(fetch_response.status_code)
    sleep(0.2) # sleep to avoid overloading the server
    fetch_root = ET.fromstring(fetch_response.content)
    for i in fetch_root.findall("Id"):
        ids.add(i.text)

print("Number of ids: ", len(ids))
print("Expected number of ids: ", count)

# pprint.pprint(ids)


Number of ids:  3326542
Expected number of ids:  3326542


In [126]:
returned_ids = set(filtered_results.keys())
print("Number of ids with 'blood' in the BTO: ", len(returned_ids))

# ids that have returned results but not in the NCBI search - False Positives
set_difference = returned_ids.difference(ids)

print(len(set_difference), "False Positives")
print(set_difference)

print('33544736' in ids)

Number of ids with 'blood' in the BTO:  779
94 False Positives
{'832160', '16016553', '33544736', '33504971', '517051', '1157998', '12191993', '528273', '2023613', '34148567', '2271963', '865526', '2282655', '3675248', '519689', '6943924', '13691856', '911242', '934081', '13693299', '29117118', '13626487', '33520202', '1864579', '2278320', '865590', '12197483', '2276754', '784669', '2014880', '16017558', '2016366', '2683459', '3671428', '302869', '2266438', '2460115', '519059', '519521', '218984', '949659', '33543813', '2286190', '979049', '33539189', '909081', '2683759', '524175', '984728', '910382', '520440', '16033835', '2452554', '12193874', '831042', '2021401', '33517707', '3679727', '9827805', '2171996', '2017151', '2159929', '2447029', '22579999', '6944214', '529484', '916604', '2446356', '1925881', '945679', '922416', '8855655', '2271410', '1183519', '2282069', '2010857', '2292257', '6133933', '1223953', '6945860', '528239', '2018405', '9826181', '2276390', '2268649', '12800143